In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import geopandas as gpd
import rioxarray
import json
import rasterio
from rioxarray.merge import merge_arrays
from shapely.geometry import Polygon
import warnings


warnings.filterwarnings("ignore")
pd.set_option("max_colwidth", None)

In [2]:
# define a function to extract the landcover data to a geopandas dataframe
def extract_landcover_data(RCP="RCP26", model="GFDL", year= '2050'):
    # get the landcover data
    data_path = "D:\hotspot mapping\Land_Use_Harmonization_V2_1721\data"

    file_path = data_path + f"\{RCP}_{model}_states.nc4"
    # Load .nc4 file
    dataset = xr.open_dataset(file_path)

    datetime = f'{year}-01-01 00:00:00'
    
    # traverse all the variables in the dataset
    gdf_list = []
    final_gdf = gpd.GeoDataFrame()
    for land in dataset.data_vars:
        if land=="crs":
            continue
        # print(land)
        df = dataset[land].sel(time=datetime).to_dataframe().reset_index()
        gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat)).drop(["lat","lon","time"], axis=1)
        # print(gdf.head())
        # print(final_gdf.head())
        if final_gdf.empty:
            final_gdf = gdf
        else:
            final_gdf = gpd.sjoin(final_gdf, gdf, how="left")
            # remove the duplicate columns
            final_gdf = final_gdf.drop(["index_right"], axis=1)
    
    # df = dataset.sel(time=datetime).to_dataframe()
    return final_gdf

In [5]:
def process_land_cover_gdf(area_gdf, area_polygon=None):
    # drop the rows with null 'urban' values
    area_gdf = area_gdf.dropna(subset=['urban'])
    # change the geometry from point to polygon
    area_gdf['geometry'] = area_gdf['geometry'].apply(lambda x: Polygon([[x.x-0.125, x.y-0.125], [x.x-0.125, x.y+0.125], [x.x+0.125, x.y+0.125], [x.x+0.125, x.y-0.125]]))
    
    # get polygon of costa rica
    if(area_polygon==None):
        url = 'https://raw.githubusercontent.com/HotspotStoplight/HotspotStoplight/main/CropBoxes/CR_Crop3.geojson'
        gdf2 = gpd.read_file(url)

        area_polygon = gdf2.geometry[0]
    
    # get the intersection of the landcover data and the area polygon
    area_gdf['intersection'] = area_gdf.intersection(area_polygon)
    area_gdf['area'] = area_gdf['intersection'].area
    # forest types
    forest_types = ['primf','secdf']
    
    # impute the null values with 0
    area_gdf['area'] = area_gdf['area'].fillna(0)
    # get the total fraction for each row
    area_gdf['forest']=area_gdf.apply(lambda x: sum(x[forest_types]), axis=1)
    area_gdf['other types'] = 1-area_gdf['forest']-area_gdf['urban']

    return area_gdf

In [3]:
gdf = extract_landcover_data()
area_gdf = process_land_cover_gdf(gdf)

type: name 'process_land_cover_gdf' is not defined

In [10]:
def get_landcover_demand_from_gdf(area_gdf):
    # calculate the demand for each landcover type
    land_cover_list = ['forest', 'urban', 'other types']
    land_cover_demand = []
    for land in land_cover_list:
        land_cover_demand.append((area_gdf[land] * area_gdf['area']).sum())
    total_area = area_gdf['area'].sum()
    land_cover_demand = [x/total_area for x in land_cover_demand]
    return land_cover_demand

In [13]:
def get_single_model_demand(RCP="RCP26", model="GFDL", year= '2050',area_polygon=None):
    gdf = extract_landcover_data(RCP, model, year)
    area_gdf = process_land_cover_gdf(gdf,area_polygon)
    land_cover_demand = get_landcover_demand_from_gdf(area_gdf)
    print("model: ", model, "demand: ", land_cover_demand)
    return land_cover_demand

In [14]:
def get_avg_demand(RCP="RCP26", year= '2050',area_polygon=None):
    models = ['GFDL', 'IPSL', 'MIROC', 'HADGEM']
    model_demands = []
    for model in models:
        model_demands.append(get_single_model_demand(RCP, model, year,area_polygon))
    # get the average demand
    avg_demand = np.mean(model_demands, axis=0)
    return avg_demand

In [19]:
def output_csv(demand_list):
    land_cover_demand_dict = dict(zip(['forest', 'urban', 'other types'], demand_list))
    # convert the dictionary to a csv file
    land_cover_demand_df = pd.DataFrame(land_cover_demand_dict, index=[0])
    # add the row key
    land_cover_demand_df.to_csv('land_cover_demand.csv', index=False, header=True)

In [20]:
avg_demand = get_avg_demand()
output_csv(avg_demand)